# Post

In [3]:
import requests

server_url = ['61.74.232.165:5000','192.168.0.178:5000', 'localhost:5000'][1]
upload_url=server_url + '/upload'
cert = False#"./cert.pem"

# 키 전달
def set_key():
    # 미리 정해져있음 
    flists = [("enc_key", 'EncKey.txt'), ('mul_key', 'MulKey.txt')]
    for dtype, fn in flists:
        r = requests.post('https://'+upload_url, 
                          files={'file':open(fn, 'rb')}, 
                          headers={'dtype':dtype},
                          verify=cert)
        print(r)
        if not r.ok:
            print("ERROR")
            return -1
    return True

# 암호문 전달
def send_ctxt(action, ori):
    """e.g., ctx_05_e.dat"""
    files = {'file': open(f'./ctx_{action.zfill(2)}_{ori}.dat','rb')}
    header = {'dtype':"ctxt", "action":f'{action}'}
    r = requests.post('https://'+upload_url, 
                      files=files, 
                      headers=header,
                      verify=cert)
    return r.ok

with open("EncKey.txt", "w") as f:
    f.write("EEEE")
    
with open("MulKey.txt", "w") as f:
    f.write("MMM")

r = set_key()
print(r)

<Response [200]>
<Response [200]>
True


/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.178'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.178'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
r = send_ctxt('13', 'a')

/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.178'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# get

서버에 계산된 암호문 파일 받은 후 동형암호 라이브러리로 읽어서 복호화, 결과 확인.
`
**서버 계산이 완료될 때 까지 5초에 한번씩 get 시도**


In [30]:
from time import sleep
from urllib.parse import unquote


def save_binary(r, fn_save):
    if r.status_code == 200:
        with open(fn_save, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        raise FileNotFoundError

result_url='https://'+server_url + '/result'

def get_result(result_url):
    n_try = 0
    while n_try < 10:
        r = requests.get(result_url, stream=True, verify=cert)
        if r.status_code == 200:
            save_binary(r,'./client_results/pred0.dat')
            print("Result recieved.")
            break
        else:
            sleep(5)
            n_try+=1
    else:
        print("Retry limit reached. Try again later")
        return False
    return r

def get_filename(response):
    if 'Content-Disposition' in response.headers:
        content_disposition = response.headers['Content-Disposition']
        parts = content_disposition.split(';')

        for part in parts:
            if 'filename' in part:
                filename = part.split('=')[1]
                filename = unquote(filename.strip(' "'))  # remove quotes and spaces
                
    return filename

In [14]:
r = get_result(result_url)

Result recieved.


/home/hoseung/anaconda3/envs/fhe/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.178'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [35]:
fn_ctx = get_filename(r)
print(fn_ctx)

pred_0.dat


### 결과 확인 예시

In [33]:
import fase
from fase import HEAAN as he

In [38]:
# HEAAN 라이브러리로 ctxt 파일 로드
logp = 30
logq = 300
nslots = 2**14
ctx_pred = he.Ciphertext(logp, logq, nslots)
he.SerializationUtils.readCiphertext(ctx_pred, fn_ctx)
# decryptor로 decrypt
dec=decrypt(self.scheme, self.secretKey, ctx_pred, self.parms)
# 결과 출력 
print(np.argmax(dec))

MemoryError: std::bad_alloc